# Data Acquisition

In [ ]:
# Import functions
import pandas as pd
import numpy as np
import math
import os
import acquire

import random
random.seed(123)

# Acquire the data

In [ ]:
# Load in primary dataset.
df = pd.read_csv('train.csv', usecols=[1,2,3,4,5,6,7,8,9])

# Training dataset
To reduce the memory size of our training data, we selected a random sample of 100,000 students from `train.csv`.
1. Select 100,000 user ids from `train.csv` to use as our train dataset.
2. Cast numeric column data types using an appropriate precision to reduce memory size.

## Sample training data
- Selecting 100_000 users.

In [ ]:
# Find all users with more than 10 rows.
user_ids = df['user_id'].value_counts()[df['user_id'].value_counts() > 10].index.to_list()

# Select a random sample of 100_000 user_ids.
sampled_ids = random.sample(user_ids, 100_000)

In [ ]:
len(sampled_ids)

In [ ]:
def sampled_users(df):
    '''
    This function accepts data from `train.csv` and
    returns a random sample of 100_000 user_ids.
    '''
    user_ids = df['user_id'].value_counts()[df['user_id'].value_counts() > 10].index.to_list()
    sampled_ids = random.sample(user_ids, 100_000)
    return sampled_ids

In [ ]:
# Using the selected user_ids, filter the dataset for the first 100_000.
filtered_data = pd.DataFrame()

data = df.loc[df['user_id'].isin(sampled_ids)]

In [ ]:
# We have 100_000 users!!!!
data.user_id.nunique()

In [ ]:
# Cache local file for next steps.
# data.to_csv('sampled_train.csv', index=False)

# Dataset loads correctly.
# df = pd.read_csv('sampled_train.csv')

# Casting data types

In [ ]:
def datatype_converter():
    '''
    This function returns a dictionary of column names and data types to convert.
    '''
    
    train_data_types_dict = {
    'timestamp': np.int64,
    'user_id': np.int32,
    'content_id': np.int16,
    'content_type_id': np.int16,
    'task_container_id' : np.int16,
    'user_answer' : np.int8,
    'answered_correctly': np.int8,
    'prior_question_elapsed_time': np.float16
    }
    
    lectures_data_types_dict = {
    'lecture_id' : np.int16,
    'tag' : np.int8,
    'part' : np.int8
    }

    questions_data_types_dict = {
    'question_id' : np.int16,
    'bundle_id' : np.int16,
    'part' : np.int8
    }
    
    
    return train_data_types_dict, lectures_data_types_dict, questions_data_types_dict

In [ ]:
def sampled_train():
    '''
    This function selects a random sample of 100_000 users from the `train.csv` dataset.
    Returns a dataframe of 100_000 users that have more than 10 rows of data.
    
    
    Parameters
    ----------
    None
    
    
    Returns
    -------
    data : pandas.core.frame.DataFrame
        A pandas dataframe of 100,000 randomly selected
        users.
    '''
    train_dtypes, _, _ = datatype_converter()
    
    if os.path.isfile('sampled_train.csv'):
        return pd.read_csv('sampled_train.csv',
                           index_col=False,
                           dtype=train_dtypes)
    else:
        
    # Read in `train.csv` data
        df = pd.read_csv('train.csv', dtype=train_dtypes, usecols=[1,2,3,4,5,6,7,8,9])

        sampled_ids = sampled_users(df)

        sampled_data = df.loc[df['user_id'].isin(sampled_ids)]

        # Cache local file of sampled data.
        sampled_data.to_csv('sampled_train.csv', index=False)
    
    # Return the dataframe
    return sampled_data

Create a function to reproduce the modified training set.

In [ ]:
# Test the function.
df = sampled_train()

In [ ]:
# It works.
df.user_id.value_counts()

In [ ]:
# Check our data types and memory usage.
df.info()

# Merge datasets together

In [ ]:
_, lecture_dtypes, question_dtypes = datatype_converter()

df_lectures = pd.read_csv('lectures.csv', dtype=lecture_dtypes)
df_questions = pd.read_csv('questions.csv', dtype=question_dtypes)

# Left join df_train and df_lectures using `content_id` as the primary key.
df_merged = df.merge(df_lectures, left_on='content_id', right_on='lecture_id', how='left')

# Left join df_merged and df_questions using `content_id` as the primary key.
df_data = df_merged.merge(df_questions, left_on='content_id', right_on='question_id', how='left')

In [ ]:
df_data.info()

In [ ]:
import acquire

In [ ]:
df = acquire.get_riiid_data()

In [ ]:
df.head()

In [ ]:
train = pd.DataFrame()
validate = pd.DataFrame()
test = pd.DataFrame()

# Set up the train size
train_size = 0.8
validate_size = 0.1

sampled_ids = df.user_id.unique()

In [ ]:
for user in sampled_ids:
    data = df.loc[df['user_id'] == user]
    n = data.shape[0]

    train_end_index = int(train_size * n)
    validate_end_index = train_end_index + int(validate_size * n)

    df_train = data.iloc[:train_end_index]
    df_validate = data.iloc[train_end_index:validate_end_index]
    df_test = data.iloc[validate_end_index:]

    train = pd.concat([train, df_train])
    validate = pd.concat([validate, df_validate])
    test = pd.concat([test, df_test])

In [ ]:
# Print the shape of the original, train and test
train.shape, validate.shape, test.shape

In [ ]:
train.to_csv('sampled_trainset.csv', index=False)
validate.to_csv('validate.csv', index=False)
test.to_csv('test.csv', index=False)

In [ ]:
type(train)

# Create a function to split data using Quasi-GroupKFold method
The way our data is currently split for MVP:
- 100,000 randomly selected users that have more that 10 interactions with Riiid's Knowledge tracing application.
- Each user has _sequential_ data, indicated by the variable `timestamp`.
- Data is split using a percentage-based method.
- 0% - 80% of a users data is the training set.
- 80% - 90% of a users data is in the validation set.
- 90% - 100% of a users data is in the test set.

The way we split the data is important. As we currently have our splits, several issues arise that impact our data exploration and modeling performance.
- Spliting the data using a percentage-based method removes questions and lectures from our training data. If a model encounters a question it has never seen before, how can it accurately model reality? We need to have all questions appear at least once in our dataset.
    - If this was _purely_ a time series problem, that would be fine.
- The training set uses 80% of a users data, this impacts our statistical analysis. If we have _all_ of a users data, we can correctly calculate population statistics from a _sample_ of users. We can then compare statistics on a user/grouped level with the population.

What is the solution?

> <strong>Splitting by users!</strong>

## Splitting by users
How does this solve our exploration and modeling issues?
> <strong>By creating users the model has never seen before!</strong>

This simulates _new_ users interacting with Riiid's Knowledge Tracing Application.

In [ ]:
df_data = acquire.get_riiid_data()

In [ ]:
df_data.lecture_id = df_data.lecture_id.astype('Int16')
df_data.tag = df_data.tag.astype('Int8')
df_data.part_x = df_data.part_x.astype('Int8')
df_data.part_y = df_data.part_y.astype('Int8')
df_data.question_id = df_data.question_id.astype('Int16')
df_data.bundle_id = df_data.bundle_id.astype('Int16')
df_data.lecture_id = df_data.lecture_id.astype('Int32')

# Prefix part names with the originating dataframe name.
df_data.rename(columns={'part_x': 'lecture_part',
                        'part_y': 'question_part'},
                   inplace=True)

# Cache the data.
df_data.to_csv('riiid_data.csv', index=False)

### How many users are in the dataset?

In [ ]:
users = df_data.user_id.unique()
print(f'There are {len(users):,} users.')

### Create new data splits using percentage of users

In [ ]:
# Percentages of orginal dataset
train_size = .8
validate_size = .1
test_size = .1

# Calculate the number of users in each dataset.
train_users = int(len(users)*train_size)
validate_users = math.ceil(len(users)*validate_size)
test_users = math.ceil(len(users)*test_size)

# Display the results.
print(f'Train set would contain {train_users:,} users')
print(f'Validate set would contain {validate_users:,} users')
print(f'Test set would contain {test_users:,} users')

In [ ]:
# Check for even splits
print(len(users) == sum([train_users, validate_users, test_users]))

print(len(users))
print(sum([train_users, validate_users, test_users]))

### Test a method to split users into seperate datasets

In [ ]:
# Set a random seed to reproduce splits
random.seed(123)

# Toy example
all_users = [1, 2, 3, 4, 5]

# Randomly select train set users
t_ids = random.sample(all_users, 3)

# Remove users assigned to the training set.
list(set(all_users) - set(t_ids))

### Create a function to reproduce data splits

In [ ]:
def split_users(df, train_size=.8, validate_size=.1, test_size=.1):
    '''
    This function accepts the merged dataframe from acquire.get_riiid_data()
    and returns train, validate and test sets.
    '''
    # Set a random seed to reproduce splits
    random.seed(123)
    
    # Gather all user ids
    user_ids = list(df['user_id'].unique())
    total_num = len(user_ids)
    
    # Calculate the number of users in train, validate, and test.
    train_num = int(total_num*train_size)
    validate_num = math.ceil(total_num*validate_size)
    test_num = math.ceil(total_num*test_size)
    
    # Randomly select 80% of the users to be in train.
    train_ids = random.sample(user_ids, train_num)
    
    # Remove user_ids assigned to the training set.
    remaining_val_test_users = list(set(user_ids) - set(train_ids))
    
    # Assign the remaining user ids to validate and test.
    validate_ids = random.sample(remaining_val_test_users, validate_num)
    test_ids = list(set(remaining_val_test_users) - set(validate_ids))
    
    # Return the users assigned to train, validate, and test
    return train_ids, validate_ids, test_ids

In [ ]:
train_ids, validate_ids, test_ids = split_users(df_data)

In [ ]:
train = df_data.loc[df_data['user_id'].isin(train_ids)]
validate = df_data.loc[df_data['user_id'].isin(validate_ids)]
test = df_data.loc[df_data['user_id'].isin(test_ids)]

In [ ]:
train.to_csv('train_k.csv', index=False)
validate.to_csv('validate_k.csv', index=False)
test.to_csv('test_k.csv', index=False)

In [ ]:
train.info()

### Modify the function to use a subset of the data

In [ ]:
def split_users(df, train_size=.8, validate_size=.1, test_size=.1, sample=True):
    '''
    This function accepts the merged dataframe from acquire.get_riiid_data()
    and returns train, validate and test sets.
    '''
    # Set a random seed to reproduce splits
    random.seed(123)
    
    if sample == True:
        # Gather a random sample of 100_000 user ids
        user_ids = random.sample(list(df['user_id'].unique()), 100_000)
    else:
        # Gather all user ids
        user_ids = list(df['user_id'].unique())
    
    # Calculate the number of users
    total_num = len(user_ids)
    
    # Calculate the number of users in train, validate. Remaining users go in test
    train_num = int(total_num*train_size)
    validate_num = math.ceil(total_num*validate_size)
    
    # Randomly select 80% of the users to be in train.
    train_ids = random.sample(user_ids, train_num)
    
    # Remove user_ids assigned to the training set.
    remaining_val_test_users = list(set(user_ids) - set(train_ids))
    
    # Assign the remaining user ids to validate and test.
    validate_ids = random.sample(remaining_val_test_users, validate_num)
    test_ids = list(set(remaining_val_test_users) - set(validate_ids))
    
    # Return the users assigned to train, validate, and test
    return train_ids, validate_ids, test_ids


def train_validate_test(df, sampled=True):
    '''

    '''
    train_ids, validate_ids, test_ids = split_users(df, sample=sampled)

    train = df.loc[df['user_id'].isin(train_ids)]
    validate = df.loc[df['user_id'].isin(validate_ids)]
    test = df.loc[df['user_id'].isin(test_ids)]

    return train, validate, test

In [ ]:
train, validate, test = train_validate_test(acquire.get_riiid_data())
train.to_csv('mvp_train.csv', index=False)
validate.to_csv('mvp_validate.csv', index=False)
test.to_csv('mvp_test.csv', index=False)

In [ ]:
train.info()

### Modify the function to use a subset of the data: Sample of 50K users

In [ ]:
def split_users(df, train_size=.8, validate_size=.1, sample=True):
    '''
    This function accepts the merged dataframe from acquire.get_riiid_data()
    and returns train, validate and test sets.
    '''
    # Set a random seed to reproduce splits
    random.seed(123)
    
    if sample == True:
        # Gather a random sample of 50_000 user ids
        user_ids = random.sample(list(df['user_id'].unique()), 50_000)
    else:
        # Gather all user ids
        user_ids = list(df['user_id'].unique())
    
    # Calculate the number of users
    total_num = len(user_ids)
    
    # Calculate the number of users in train, validate. Remaining users go in test
    train_num = int(total_num*train_size)
    validate_num = math.ceil(total_num*validate_size)
    
    # Randomly select 80% of the users to be in train.
    train_ids = random.sample(user_ids, train_num)
    
    # Remove user_ids assigned to the training set.
    remaining_val_test_users = list(set(user_ids) - set(train_ids))
    
    # Assign the remaining user ids to validate and test.
    validate_ids = random.sample(remaining_val_test_users, validate_num)
    test_ids = list(set(remaining_val_test_users) - set(validate_ids))
    
    # Return the users assigned to train, validate, and test
    return train_ids, validate_ids, test_ids


def train_validate_test(df, sampled=True):
    '''
    
    '''
    train_ids, validate_ids, test_ids = split_users(df, sample=sampled)

    train = df.loc[df['user_id'].isin(train_ids)]
    validate = df.loc[df['user_id'].isin(validate_ids)]
    test = df.loc[df['user_id'].isin(test_ids)]

    return train, validate, test

In [ ]:
train, validate, test = train_validate_test(acquire.get_riiid_data())

In [ ]:
train.to_csv('mvp_50_train.csv', index=False)
validate.to_csv('mvp_50_validate.csv', index=False)
test.to_csv('mvp_50_test.csv', index=False)

# Exploration

In [ ]:
# Import functions
import pandas as pd
import numpy as np
import math
import os
import acquire
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import random
random.seed(123)

Four features I will explore:
1. Running accuracy within a bundle
2. Question Tag Accuracy
3. Tag_#
4. Cumulative user accuracy using Exponentially Weighted Mean

In [ ]:
train = pd.read_csv('sampled_trainset.csv')
validate = pd.read_csv('validate.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
# Remove lectures rows from the training set.
train = train.loc[(train.answered_correctly != -1)]

## Part Analysis

Part1<br>
Photographs: 6 questions
>Four short statements regarding a photograph will be spoken only one time. The statements will not be printed. Of these four statements, select the one that best describes the photograph and mark your answer on the answer sheet.

Part2<br>
Question-Response: 25 questions
>Three responses to one question or statement will be spoken only one time. They will not be printed. Select the best response for the question, and mark your answer on the answer sheet.

Part3<br>
Conversations: 39 questions
>Conversations between two or three people will be spoken only one time. They will not be printed. Listen to each conversation and read the questions printed in the test book (the questions will also be spoken), select the best response for the question, and mark your answer on the answer sheet. Some questions may require responses related to information found in diagrams,etc. printed on the test book as well as what you heard in the conversations. There are three questions for each conversation.

Part4<br>
Talks: 30 questions
>Short talks such as announcements or narrations will be spoken only one time. They will not be printed. Listen to each talk and read the questions printed in the test book (the questions will also be spoken), select the best response for the question, and mark your answer on the answer sheet. Some questions may require responses related to information found in diagrams, etc. printed on the test book as well as what you heard in the talks. There are three questions for each talk.

Reading Section (75 minutes, 100 questions): Read printed questions and answer them.<br>
Part5<br>
Incomplete Sentences: 30 questions
> Select the best answer of the four choices to complete the sentence, and mark your answer on the answer sheet.

Part6<br>
Text Completion: 16 questions
>Select the best answer of the four choices (words, phrases, or a sentence) to complete the text, and mark your answer on the answer sheet. There are four questions for each text.

Part7<br>
Single Passages: 29 questions
Multiple Passages: 25 questions
> A range of different texts will be printed in the test book. Read the questions, select the best answer of the four choices, and mark your answer on the answer sheet. Some questions may require you to select the best place to insert a sentence within a text. There are multiple questions for each text. 

In [ ]:
sns.set_context('talk')

pd.crosstab(train.question_part, train.answered_correctly).plot.bar(figsize=(13, 7))

plt.legend().set_visible(False)
plt.title('How many user interactions are there for each part of the exam?')
plt.xlabel('Part of Exam')
plt.ylabel('Number of Questions (in Millions)')
plt.xticks(np.arange(0, 7), labels=['1', '2', '3', '4', '5', '6', '7'], rotation=0)
plt.yticks([1_000_000, 2_000_000, 3_000_000, 4_000_000, 5_000_000], labels=['1', '2', '3', '4', '5'], rotation=0);

In [ ]:
sns.set_context('talk')

pd.crosstab(train.question_part, train.answered_correctly, normalize='index').plot.bar(figsize=(13, 7))
plt.legend().set_visible(False)
plt.title('How often do users get questions correct for each part of the exam?')
plt.xlabel('Part of Exam')
plt.ylabel('Percentage')
plt.xticks(np.arange(0, 7), labels=['1', '2', '3', '4', '5', '6', '7'], rotation=0)
plt.ylim(0, 1);

## Exponentially Weighted Mean of a users accuracy

In [ ]:
user = train.loc[train.user_id == 24600]

In [ ]:
user.head()

In [ ]:
# The culmulative total of questions answered correctly by a single user.
user.answered_correctly.cumsum()

In [ ]:
# Number of questions a single user has answered
len(user)

In [ ]:
user.answered_correctly.cumsum()/range(1, len(user.answered_correctly)+1)

In [ ]:
# Calculating the cumulative accuracy of a single user
plt.scatter(x=range(0, len(user.timestamp)),
            y=user.answered_correctly.cumsum()
              / range(1, len(user.answered_correctly)+1))

In [ ]:
user.answered_correctly

In [ ]:
user.answered_correctly.ewm(3).mean().plot()
user.answered_correctly.ewm(7).mean().plot()
user.answered_correctly.ewm(10).mean().plot()

In [ ]:
set_user = 0
counter = 0

for index, row in train[:5].iterrows():
    user = row['user_id']
    if set_user == 0:
        current_user = row['user_id']
        counter += 1
    elif user != current_user:
        counter = 0
    elif user == current_user:
        counter += 1
        
print(counter)

In [ ]:
data = pd.DataFrame()

for id_ in train.user_id.unique()[:10]:
    user_data = train['answered_correctly'].loc[train.user_id == id_]
    id_num = train['user_id'].loc[train.user_id == id_]
    user_ewm = pd.concat([id_num, user_data.ewm(3).mean()], axis=1)
    data = pd.concat([data, user_ewm])

In [ ]:
data.answered_correctly.plot()

In [ ]:
for user in data.user_id.unique():
    data['answered_correctly'].loc[train.user_id == user].plot()

## Check to see the number of unique values in each column

In [ ]:
for column in train.columns:
    print(column, train[f'{column}'].nunique())

## Bundle ID Accuracy

In [ ]:
# The accuracy of a user for each bundle they've taken.
user.groupby(['bundle_id'])['answered_correctly'].mean().to_frame()

In [ ]:
train.groupby(['bundle_id'])['answered_correctly'].mean().iloc[:10].plot.bar();

In [ ]:
bundle_accuracy = train.groupby(['bundle_id'])['answered_correctly'].mean().to_frame().reset_index()
bundle_accuracy.columns = ['bundle_id', 'bundle_mean_accuracy']
merged_train = train.merge(bundle_accuracy, left_on='bundle_id', right_on='bundle_id', how='left')

## Part Accuracy

In [ ]:
tag_accuracy = train.groupby(['question_part'])['answered_correctly'].agg(['mean']).reset_index()
tag_accuracy.columns = ['question_part', 'part_accuracy']
train_df = merged_train.merge(tag_accuracy, left_on='question_part', right_on='question_part')
train_df.head()

## Part Container Accuracy

In [ ]:
mean_part_task_container = train.groupby(['question_part', 'task_container_id'])['answered_correctly'].mean().reset_index()

In [ ]:
mean_part_task_container.head()

In [ ]:
mean_part_task_container.loc[mean_part_task_container.question_part == 2.0].head()

In [ ]:
print('Number of containers in part 1', len(mean_part_task_container.loc[mean_part_task_container.question_part == 1.0]))
print('Number of containers in part 2', len(mean_part_task_container.loc[mean_part_task_container.question_part == 2.0]))

In [ ]:
tag_bundles = train.groupby(['question_id', 'task_container_id', 'question_part'])['answered_correctly'].mean().reset_index()
tag_bundles.rename(columns={'answered_correctly': 'mean_container_part_accuracy'}, inplace=True)

In [ ]:
tag_bundles.head()

In [ ]:
plt.figure(figsize=(13, 7))
for i in range(1, 8):
    part_data = tag_bundles.loc[tag_bundles.question_part == float(i)]
    sns.ecdfplot(data=part_data, x="mean_container_part_accuracy")

In [ ]:
train.merge(tag_bundles, how='left', left_on='random'

In [ ]:
train_df.head()

## Create a function to reproduce feature engineering

In [ ]:
train.head()

In [ ]:
def tag_bundle_features(train, validate, test):
    '''
    This function accepts the train, validate, and test sets.
    Returns new features on train, validate, and test using population stats from the training set.
    
    Parameters
    ----------
    
    
    Returns
    -------
    
    '''
    
    # Calculate the average accuracy for each unique bundle id
    bundle_accuracy = train.groupby(['bundle_id'])['answered_correctly'].mean().to_frame().reset_index()
    bundle_accuracy.columns = ['bundle_id', 'mean_bundle_accuracy']
    
    # Add bundle mean accuracy as a feature to train, validate, and test
    merged_train = train.merge(bundle_accuracy, left_on='bundle_id', right_on='bundle_id', how='left')
    merged_validate = validate.merge(bundle_accuracy, left_on='bundle_id', right_on='bundle_id', how='left')
    merged_test = test.merge(bundle_accuracy, left_on='bundle_id', right_on='bundle_id', how='left')
    
    # Calculate the average part accuracy
    tag_accuracy = train.groupby(['question_part'])['answered_correctly'].agg(['mean']).reset_index()
    tag_accuracy.columns = ['question_part', 'mean_part_accuracy']
    
    # Add average part accuracy
    train_df = merged_train.merge(tag_accuracy, left_on='question_part', right_on='question_part')
    validate_df = merged_validate.merge(tag_accuracy, left_on='question_part', right_on='question_part')
    test_df = merged_test.merge(tag_accuracy, left_on='question_part', right_on='question_part')
    
    # Calculate the mean container accuracy for each part
    tag_bundles = train.groupby(['question_id', 'task_container_id', 'question_part'])['answered_correctly'].mean().reset_index()
    tag_bundles.rename(columns={'answered_correctly': 'mean_container_part_accuracy'}, inplace=True)
    
    # Add mean container part accuracy
    train_df = train_df.merge(tag_bundles)
    validate_df = validate_df.merge(tag_bundles)
    test_df = test_df.merge(tag_bundles)

    
    return train_df, validate_df, test_df

In [ ]:
train, validate, test = tag_bundle_features(train, validate, test)

In [ ]:
train.head()

## Correlations between the features and the target variable

In [ ]:
train.corr()['answered_correctly'].sort_values(ascending=False)

# Hypothesis Testing
---
## Hypothesis Test
### Question accuracy is dependent on the type of question asked.

Test: Chi2 Test<br>
$H_0$ Whether a user answers a question correctly is independent of the type of question being asked.<br>
$H_a$ Whether a user answers a question correctly is dependent upon the type of question being asked.

In [ ]:
confidence_interval = 0.95 
alpha = 1 - alpha

# Contingency table
table = pd.crosstab(train_df.answered_correctly, train_df.question_part)
chi2, p, dof, expected = stats.chi2_contingency(table)


if p < alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
    
print(p)

- P is less than alpha so we <strong>reject</strong> the null hypothesis that answering a question correctly is independent of the type of question being asked. (Different parts of the TOEIC exam)

- The 7 parts of the TOEIC exam require the user to answer questions with different formats: Pictures, Listening to conversations, Reading Conversations, Filling in Incomplete Sentences, etc. Depending on which part the user is answering questions...

In [1]:
# Import functions
import pandas as pd
import numpy as np
import math
import os
import acquire
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import random
random.seed(123)

import prepare
import explore

In [2]:
train = pd.read_csv('sampled_trainset.csv')
validate = pd.read_csv('validate.csv')
test = pd.read_csv('test.csv')

In [ ]:
train, validate, test = prepare.prep_riiid(train, validate, test)